# MVP E-commerce: Análise de Comportamento de Compras

## 1. Objetivo
Este projeto tem como objetivo analisar de forma macro a compra de clientes em um e-commerce com base no dataset Online Retail da UCI. Através da construção de um pipeline de dados em camadas (Bronze, Silver e Gold), serão respondidas as seguintes perguntas de negócio:

- Quais são os 10 países que mais representam em faturamento?
- Quais são os 10 países que mais colocaram pedidos?
- Qual o ticket médio por cliente?
- Quais são os 5 clientes que mais colocaram pedidos e onde eles ficam?

In [0]:
# Imports
import pandas as pd
import io
import requests

In [0]:
#mport pandas as pd

url = "https://raw.githubusercontent.com/gu055805/MVP/main/Online_Retail_FULL.csv.gz"

# Lê o arquivo CSV compactado diretamente do GitHub
df = pd.read_csv(url, compression='gzip')

# Visualiza as primeiras linhas
df.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [0]:

%sql
DROP DATABASE IF EXISTS bronze CASCADE;
CREATE DATABASE bronze;


In [0]:

retail_spark_df = spark.createDataFrame(df)
retail_spark_df.write.mode("overwrite").saveAsTable("bronze.online_retail")

In [0]:
%sql
SELECT * FROM bronze.online_retail LIMIT 10;

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/2010 8:26,7.65,17850.0,United Kingdom
536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/2010 8:26,4.25,17850.0,United Kingdom
536366,22633,HAND WARMER UNION JACK,6,12/1/2010 8:28,1.85,17850.0,United Kingdom
536366,22632,HAND WARMER RED POLKA DOT,6,12/1/2010 8:28,1.85,17850.0,United Kingdom
536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/2010 8:34,1.69,13047.0,United Kingdom


In [0]:
%sql
DROP DATABASE IF EXISTS silver CASCADE;
CREATE DATABASE silver;

In [0]:
%sql
SELECT StockCode, Description
FROM bronze.online_retail
WHERE StockCode IS NOT NULL
LIMIT 10;



StockCode,Description
21539,RED RETROSPOT BUTTER DISH
21408,SPOTTY PINK DUCK DOORSTOP
21890,S/6 WOODEN SKITTLES IN COTTON BAG
20977,36 PENCILS TUBE WOODLAND
37448,CERAMIC CAKE DESIGN SPOTTED MUG
21890,S/6 WOODEN SKITTLES IN COTTON BAG
22198,LARGE POPCORN HOLDER
21210,SET OF 72 RETROSPOT PAPER DOILIES
21657,MILK BOTTLE WITH GLASS STOPPER
85114C,RED ENCHANTED FOREST PLACEMAT


In [0]:
%sql
SELECT
  COUNT(*) AS total,
  COUNT(*) - COUNT(InvoiceNo) AS nulos_InvoiceNo,
  COUNT(*) - COUNT(StockCode) AS nulos_StockCode,
  COUNT(*) - COUNT(Description) AS nulos_Description,
  COUNT(*) - COUNT(Quantity) AS nulos_Quantity,
  COUNT(*) - COUNT(InvoiceDate) AS nulos_InvoiceDate,
  COUNT(*) - COUNT(UnitPrice) AS nulos_UnitPrice,
  COUNT(*) - COUNT(CustomerID) AS nulos_CustomerID,
  COUNT(*) - COUNT(Country) AS nulos_Country
FROM bronze.online_retail;



total,nulos_InvoiceNo,nulos_StockCode,nulos_Description,nulos_Quantity,nulos_InvoiceDate,nulos_UnitPrice,nulos_CustomerID,nulos_Country
541909,0,0,1454,0,0,0,135080,0


In [0]:


dbutils.fs.rm("dbfs:/user/hive/warehouse/silver.db/dim_produto", recurse=True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/silver.db/dim_cliente", recurse=True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/silver.db/dim_tempo", recurse=True);


In [0]:
%sql
CREATE TABLE silver.dim_produto
USING DELTA
AS
SELECT DISTINCT StockCode, Description
FROM bronze.online_retail
WHERE StockCode IS NOT NULL;


num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE or REPLACE TABLE silver.dim_cliente AS
SELECT DISTINCT CustomerID, Country
FROM bronze.online_retail
WHERE CustomerID <> '' OR NOT TRIM(StockCode) = '';





num_affected_rows,num_inserted_rows


In [0]:
##%sql
##SELECT 
  ##InvoiceDate,
  ##TO_TIMESTAMP(InvoiceDate, 'd/M/yyyy H:mm') AS DataHora,
  ##YEAR(TO_TIMESTAMP(InvoiceDate, 'd/M/yyyy H:mm')) AS Ano,
  ##MONTH(TO_TIMESTAMP(InvoiceDate, 'd/M/yyyy H:mm')) AS Mes,
  ##DAY(TO_TIMESTAMP(InvoiceDate, 'd/M/yyyy H:mm')) AS Dia,
  ####DAYOFWEEK(TO_TIMESTAMP(InvoiceDate, 'd/M/yyyy H:mm')) AS DiaSemana
####FROM bronze.online_retail
##IMIT 10;



In [0]:
%sql
CREATE TABLE silver.dim_tempo AS
SELECT DISTINCT 
 InvoiceDate,
  TO_TIMESTAMP(InvoiceDate, 'd/M/yyyy H:mm') AS DataHora,
  YEAR(TO_TIMESTAMP(InvoiceDate, 'd/M/yyyy H:mm')) AS Ano,
  MONTH(TO_TIMESTAMP(InvoiceDate, 'd/M/yyyy H:mm')) AS Mes,
  DAY(TO_TIMESTAMP(InvoiceDate, 'd/M/yyyy H:mm')) AS Dia,
  DAYOFWEEK(TO_TIMESTAMP(InvoiceDate, 'd/M/yyyy H:mm')) AS DiaSemana
FROM bronze.online_retail;

num_affected_rows,num_inserted_rows


In [0]:

%sql
CREATE TABLE silver.fato_vendas AS
SELECT InvoiceNo, StockCode, CustomerID, Quantity, InvoiceDate, UnitPrice,
       Quantity * UnitPrice AS ValorTotal
FROM bronze.online_retail
WHERE CustomerID IS NOT NULL;

num_affected_rows,num_inserted_rows


In [0]:

%sql
DROP DATABASE IF EXISTS gold CASCADE;
CREATE DATABASE gold;

In [0]:

%sql
CREATE TABLE gold.vendas_flat AS
SELECT f.*, c.Country, p.Description,
       t.Ano, t.Mes, t.Dia, t.DiaSemana
FROM silver.fato_vendas f
JOIN silver.dim_cliente c ON f.CustomerID = c.CustomerID
JOIN silver.dim_produto p ON f.StockCode = p.StockCode
JOIN silver.dim_tempo t ON f.InvoiceDate = t.InvoiceDate;

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM gold.vendas_flat LIMIT 5000;

InvoiceNo,StockCode,CustomerID,Quantity,InvoiceDate,UnitPrice,ValorTotal,Country,Description,Ano,Mes,Dia,DiaSemana
536365,85123A,17850.0,6,12/1/2010 8:26,2.55,15.299999999999999,United Kingdom,CREAM HANGING HEART T-LIGHT HOLDER,2010,1,12,3
536365,85123A,17850.0,6,12/1/2010 8:26,2.55,15.299999999999999,United Kingdom,wrongly marked carton 22804,2010,1,12,3
536365,85123A,17850.0,6,12/1/2010 8:26,2.55,15.299999999999999,United Kingdom,?,2010,1,12,3
536365,85123A,17850.0,6,12/1/2010 8:26,2.55,15.299999999999999,United Kingdom,WHITE HANGING HEART T-LIGHT HOLDER,2010,1,12,3
536365,71053,17850.0,6,12/1/2010 8:26,3.39,20.34,United Kingdom,WHITE MOROCCAN METAL LANTERN,2010,1,12,3
536365,71053,17850.0,6,12/1/2010 8:26,3.39,20.34,United Kingdom,WHITE METAL LANTERN,2010,1,12,3
536365,84406B,17850.0,8,12/1/2010 8:26,2.75,22.0,United Kingdom,?,2010,1,12,3
536365,84406B,17850.0,8,12/1/2010 8:26,2.75,22.0,United Kingdom,incorrectly made-thrown away.,2010,1,12,3
536365,84406B,17850.0,8,12/1/2010 8:26,2.75,22.0,United Kingdom,null,2010,1,12,3
536365,84406B,17850.0,8,12/1/2010 8:26,2.75,22.0,United Kingdom,CREAM CUPID HEARTS COAT HANGER,2010,1,12,3


In [0]:
%sql
SELECT CustomerID, ROUND(SUM(ValorTotal)/COUNT(DISTINCT InvoiceNo), 2) AS TicketMedio
FROM gold.vendas_flat
GROUP BY CustomerID
ORDER BY TicketMedio DESC
LIMIT 10;




CustomerID,TicketMedio
15098.0,23901.6
12798.0,16150.72
15749.0,12976.6
12357.0,7809.45
12415.0,7570.96
12688.0,6979.78
12435.0,6876.45
12752.0,6743.1
17450.0,6345.41
13135.0,6192.0


In [0]:

%sql
SELECT DISTINCT Country, SUM(Quantity) AS TotalComprado
FROM gold.vendas_flat 
GROUP BY Country
ORDER BY TotalComprado DESC
LIMIT 10;

Country,TotalComprado
United Kingdom,6050405
Netherlands,295763
EIRE,199424
France,170380
Germany,163962
Australia,125153
Japan,55301
Sweden,45592
Spain,44036
Switzerland,39477


In [0]:
%sql
SELECT DISTINCT Country, COUNT(DISTINCT InvoiceNo) AS N_Pedidos
FROM gold.vendas_flat 
GROUP BY Country
ORDER BY  N_Pedidos DESC
LIMIT 10;

Country,N_Pedidos
United Kingdom,19857
Germany,603
France,458
EIRE,319
Belgium,137
Spain,118
Netherlands,101
Switzerland,76
Australia,72
Portugal,70


In [0]:
%sql
SELECT 
  CustomerID,
  Country,
  COUNT(DISTINCT InvoiceNo) AS ComprasRealizadas,
   SUM(Quantity) AS TotalComprado
FROM gold.vendas_flat
GROUP BY CustomerID, Country
ORDER BY ComprasRealizadas DESC
LIMIT 5;


CustomerID,Country,ComprasRealizadas,TotalComprado
14911.0,EIRE,248,112450
12748.0,United Kingdom,224,39355
17841.0,United Kingdom,169,31852
14606.0,United Kingdom,128,8991
13089.0,United Kingdom,118,45283


In [0]:
%sql
SELECT 
  ROUND(AVG(TotalMensal), 2) AS MediaFaturamentoMensal
FROM (
  SELECT 
    Ano, 
    Mes, 
    SUM(ValorTotal) AS TotalMensal
  FROM gold.vendas_flat
  WHERE Mes IS NOT NULL
  GROUP BY Ano, Mes
);


MediaFaturamentoMensal
264004.54
